In [1]:
import os
import numpy as np
import pickle
import h5py
#import pandas as pd

from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D



In [2]:
classes = 100
current_path = os.path.join(os.getcwd(), 'current_model')
#print(current_path)
#model_name = 'cifar100.h5'
(x_train, y_train) , (x_test, y_test) = cifar100.load_data()
print('x_train_dims : ' , x_train.shape)
print('x_test_dims : ', x_test.shape)
print('y_train_dims : ', y_train.shape)
print('y_test_dims : ', y_test.shape)

print( 'number of training examples available : ', x_train.shape[0])
print('number of testing examples available : ', x_test.shape[0])

y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0

169009152/169001437 [==============================] - 4s 0us/step
x_train_dims :  (50000, 32, 32, 3)
x_test_dims :  (10000, 32, 32, 3)
y_train_dims :  (50000, 1)
y_test_dims :  (10000, 1)
number of training examples available :  50000
number of testing examples available :  10000


In [3]:
model = Sequential()
# layer one
model.add(Conv2D(128,(3,3), kernel_initializer='he_uniform', padding = 'same', input_shape = x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), kernel_initializer='he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#layer two
model.add(Conv2D(256, (3,3), kernel_initializer='he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), kernel_initializer='he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#layer three
model.add(Conv2D(512, (3,3), kernel_initializer='he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), kernel_initializer='he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(1024, kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(classes))
model.add(Activation("softmax"))

adam=keras.optimizers.Adam(lr=.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics = ['accuracy'])


DATA AUGMENTATION

In [4]:
batch_size = 64
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
it_train = datagen.flow(x_train, y_train, batch_size=batch_size)
steps = int(x_train.shape[0] / batch_size)

In [5]:
epochs = 10

model.fit(it_train, steps_per_epoch=steps, epochs = epochs)

Epoch 1/10
781/781 [==============================] - 67s 42ms/step - loss: 4.5927 - accuracy: 0.0858
Epoch 2/10
781/781 [==============================] - 33s 42ms/step - loss: 3.3490 - accuracy: 0.2257
Epoch 3/10
781/781 [==============================] - 33s 42ms/step - loss: 2.6858 - accuracy: 0.3327
Epoch 4/10
781/781 [==============================] - 32s 41ms/step - loss: 2.3066 - accuracy: 0.3954
Epoch 5/10
781/781 [==============================] - 32s 41ms/step - loss: 2.0264 - accuracy: 0.4569
Epoch 6/10
781/781 [==============================] - 32s 41ms/step - loss: 1.8449 - accuracy: 0.4992
Epoch 7/10
781/781 [==============================] - 32s 41ms/step - loss: 1.7070 - accuracy: 0.5251
Epoch 8/10
781/781 [==============================] - 32s 41ms/step - loss: 1.5847 - accuracy: 0.5553
Epoch 9/10
781/781 [==============================] - 32s 41ms/step - loss: 1.4795 - accuracy: 0.5859
Epoch 10/10
781/781 [==============================] - 33s 42ms/step - loss: 1.390

In [6]:
test_loss, test_acc = model.evaluate(x_test,  y_test)
print(test_acc)

313/313 [==============================] - 3s 8ms/step - loss: 1.4591 - accuracy: 0.5927
0.5927000045776367


**RESNET**

In [7]:
tlEpochs=40
res_net = ResNet50(weights='imagenet', include_top=False, input_shape=x_train.shape[1:], classes=y_train.shape[1])
res_net.trainable = False
global_average_layer = GlobalAveragePooling2D()
output_layer = Dense(units=classes, activation='softmax')
tl_model = Sequential([
  res_net,
  global_average_layer,
  output_layer
])
tl_model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics = ['accuracy'])
tl_model.fit(it_train, steps_per_epoch=steps, epochs = tlEpochs)


94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/40
781/781 [==============================] - 30s 34ms/step - loss: 5.0259 - accuracy: 0.0252
Epoch 2/40
781/781 [==============================] - 26s 33ms/step - loss: 4.4013 - accuracy: 0.0522
Epoch 3/40
781/781 [==============================] - 26s 33ms/step - loss: 4.3133 - accuracy: 0.0647
Epoch 4/40
781/781 [==============================] - 26s 34ms/step - loss: 4.2605 - accuracy: 0.0726
Epoch 5/40
781/781 [==============================] - 26s 33ms/step - loss: 4.2263 - accuracy: 0.0741
Epoch 6/40
781/781 [==============================] - 26s 33ms/step - loss: 4.1755 - accuracy: 0.0819
Epoch 7/40
781/781 [==============================] - 26s 33ms/step - loss: 4.1826 - accuracy: 0.0801
Epoch 8/40
781/781 [==============================] - 26s 34ms/step - loss: 4.1488 - accuracy: 0.0849
Epoch 9/40
781/781 [==============================] - 26s 33ms/step - loss: 4.1192 - accuracy: 0.0886
Epoch 10/40
781/7

In [8]:
restest_loss, restest_acc = tl_model.evaluate(x_test,  y_test)
print(restest_acc)

313/313 [==============================] - 5s 13ms/step - loss: 3.8827 - accuracy: 0.1381
0.13809999823570251
